In [ ]:
# install missing package in the notebook environment
%pip install pandas

import pandas as pd

orders = pd.read_csv(r"D:\vs_code\GUVI\blinkit_project\data_raw\Blinkit - blinkit_orders.csv")
order_items = pd.read_csv(r"D:\vs_code\GUVI\blinkit_project\data_raw\Blinkit - blinkit_order_items.csv")
products = pd.read_csv(r"D:\vs_code\GUVI\blinkit_project\data_raw\Blinkit - blinkit_products.csv")
customers = pd.read_csv(r"D:\vs_code\GUVI\blinkit_project\data_raw\Blinkit - blinkit_customers.csv")
marketing = pd.read_csv(r"D:\vs_code\GUVI\blinkit_project\data_raw\Blinkit - blinkit_marketing_performance.csv")
feedback = pd.read_csv(r"D:\vs_code\GUVI\blinkit_project\data_raw\Blinkit - blinkit_customer_feedback.csv")

orders.head()


Check data types

In [9]:
orders.info()
customers.info()
products.info()
marketing.info()
feedback.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   order_id                5000 non-null   int64  
 1   customer_id             5000 non-null   int64  
 2   order_date              5000 non-null   object 
 3   promised_delivery_time  5000 non-null   object 
 4   actual_delivery_time    5000 non-null   object 
 5   delivery_status         5000 non-null   object 
 6   order_total             5000 non-null   float64
 7   payment_method          5000 non-null   object 
 8   delivery_partner_id     5000 non-null   int64  
 9   store_id                5000 non-null   int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 390.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------

Check missing values

In [10]:
orders.isna().sum()
marketing.isna().sum()
feedback.isna().sum()


feedback_id          0
order_id             0
customer_id          0
rating               0
feedback_text        0
feedback_category    0
sentiment            0
feedback_date        0
dtype: int64

Convert date columns to proper datetime

In [11]:
orders["order_date"] = pd.to_datetime(orders["order_date"])
marketing["date"] = pd.to_datetime(marketing["date"])
feedback["feedback_date"] = pd.to_datetime(feedback["feedback_date"])


In [12]:
orders["order_date"].head()


0   2024-07-17 08:34:01
1   2024-05-28 13:14:29
2   2024-09-23 13:07:12
3   2023-11-24 16:16:56
4   2023-11-20 05:00:39
Name: order_date, dtype: datetime64[ns]

Create daily revenue table from orders

In [16]:
import pandas as pd
import numpy as np

# If not already done:
orders["order_date"] = pd.to_datetime(orders["order_date"])

# Create a pure date column
orders["order_date_only"] = orders["order_date"].dt.date

orders[["order_id", "order_date", "order_date_only"]].head()


,order_id,order_date,order_date_only
0,1961864118,2024-07-17 08:34:01,2024-07-17
1,1549769649,2024-05-28 13:14:29,2024-05-28
2,9185164487,2024-09-23 13:07:12,2024-09-23
3,9644738826,2023-11-24 16:16:56,2023-11-24
4,5427684290,2023-11-20 05:00:39,2023-11-20


Aggregate revenue per day

In [18]:
daily_revenue = (
    orders
    .groupby("order_date_only")
    .agg(
        total_revenue = ("order_total", "sum"),
        total_orders  = ("order_id", "nunique")
    )
    .reset_index()
)

daily_revenue.head()


,order_date_only,total_revenue,total_orders
0,2023-03-16,16066.04,9
1,2023-03-17,10818.65,6
2,2023-03-18,18575.53,7
3,2023-03-19,15214.11,7
4,2023-03-20,11312.74,4


Create daily marketing table from marketing

In [19]:
marketing["date"] = pd.to_datetime(marketing["date"])
marketing["date_only"] = marketing["date"].dt.date

marketing[["date", "channel", "spend", "impressions"]].head()


,date,channel,spend,impressions
0,2024-11-05,App,1431.85,3130
1,2024-11-05,App,4506.34,3925
2,2024-11-05,Email,4524.23,7012
3,2024-11-05,SMS,3622.79,1115
4,2024-11-05,Email,2888.99,7172


In [20]:
daily_marketing = (
    marketing
    .groupby("date_only")
    .agg(
        total_spend       = ("spend", "sum"),
        total_impressions = ("impressions", "sum")
        # If you have Clicks column, you can add:
        # total_clicks      = ("Clicks", "sum")
    )
    .reset_index()
)

daily_marketing.head()


,date_only,total_spend,total_impressions
0,2023-03-17,25626.10,62246
1,2023-03-18,27163.09,51769
2,2023-03-19,27906.33,55813
3,2023-03-20,31064.98,38096
4,2023-03-21,27017.05,52805


Join Marketing + Revenue on Date

In [21]:
master_daily = pd.merge(
    daily_marketing,
    daily_revenue,
    how="outer",
    left_on="date_only",
    right_on="order_date_only"
)


In [22]:
# Prefer a single 'date' column
master_daily["date"] = master_daily["date_only"].fillna(master_daily["order_date_only"])

# Drop helper columns
master_daily = master_daily.drop(columns=["date_only", "order_date_only"])

# Sort by date
master_daily = master_daily.sort_values("date").reset_index(drop=True)

master_daily.head()


,total_spend,total_impressions,total_revenue,total_orders,date
0,NaN,NaN,16066.04,9.0,2023-03-16
1,25626.10,62246.0,10818.65,6.0,2023-03-17
2,27163.09,51769.0,18575.53,7.0,2023-03-18
3,27906.33,55813.0,15214.11,7.0,2023-03-19
4,31064.98,38096.0,11312.74,4.0,2023-03-20


Handle Zero Spend / Zero Sales & Calculate ROAS

In [23]:
master_daily["total_spend"]   = master_daily["total_spend"].fillna(0)
master_daily["total_revenue"] = master_daily["total_revenue"].fillna(0)
master_daily["total_orders"]  = master_daily["total_orders"].fillna(0)
master_daily["total_impressions"] = master_daily["total_impressions"].fillna(0)


Create flags for no-spend / no-sales days

In [24]:
master_daily["no_spend_flag"] = (master_daily["total_spend"] == 0).astype(int)
master_daily["no_sales_flag"] = (master_daily["total_revenue"] == 0).astype(int)


Calculate numeric ROAS safely

In [25]:
# Avoid division by zero
master_daily["roas"] = np.where(
    master_daily["total_spend"] > 0,
    master_daily["total_revenue"] / master_daily["total_spend"],
    np.nan  # spend = 0 → ROAS is undefined/infinite, we'll treat separately
)

# Optional: a text label to show meaning
def roas_label(row):
    if row["total_spend"] == 0 and row["total_revenue"] > 0:
        return "Infinite (no spend)"
    elif row["total_spend"] > 0 and row["total_revenue"] == 0:
        return "0 (no sales)"
    elif row["total_spend"] == 0 and row["total_revenue"] == 0:
        return "No spend & no sales"
    else:
        return f"{row['roas']:.2f}x"

master_daily["roas_label"] = master_daily.apply(roas_label, axis=1)

master_daily.head()


,total_spend,total_impressions,total_revenue,total_orders,date,no_spend_flag,no_sales_flag,roas,roas_label
0,0.00,0.0,16066.04,9.0,2023-03-16,1,0,NaN,Infinite (no spend)
1,25626.10,62246.0,10818.65,6.0,2023-03-17,0,0,0.422173,0.42x
2,27163.09,51769.0,18575.53,7.0,2023-03-18,0,0,0.683852,0.68x
3,27906.33,55813.0,15214.11,7.0,2023-03-19,0,0,0.545185,0.55x
4,31064.98,38096.0,11312.74,4.0,2023-03-20,0,0,0.364164,0.36x


Add Operations Features (Is_Late)

In [29]:
orders["promised_delivery_time"] = pd.to_datetime(orders["promised_delivery_time"])
orders["actual_delivery_time"]   = pd.to_datetime(orders["actual_delivery_time"])

orders["delay_minutes"] = (orders["actual_delivery_time"] - orders["promised_delivery_time"]).dt.total_seconds() / 60

orders["is_late"] = (orders["delay_minutes"] > 0).astype(int)

daily_ops = (
    orders
    .groupby("order_date_only")
    .agg(
        avg_delay_minutes = ("delay_minutes", "mean"),
        late_order_rate   = ("is_late", "mean")
    )
    .reset_index()
)

# Merge into master
master_daily = master_daily.merge(
    daily_ops,
    how="left",
    left_on="date",
    right_on="order_date_only"
).drop(columns=["order_date_only"])


Save the Master Dataset

In [30]:
master_daily.to_csv(r"D:\vs_code\GUVI\blinkit_project\data_processed\master_daily_roas.csv", index=False)


  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/f7/4e/510db49dd89fc3a6e994bee51848c94c48c4a00dc905e8d0133c251f41a7/sqlalchemy-2.0.45-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/b1/d2/99b55e85832ccde77b211738ff3925a5d73ad183c0b37bcbbe5a8ff04978/psycopg2_binary-2.9.11-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for greenlet>=1 from https://files.pythonhosted.org/packages/6c/79/3912a94cf27ec503e51ba493692d6db1e3cd8ac7ac52b0b47c8e33d7f4f9/greenlet-3.3.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.6.0 from https://files.pythonhosted.org/packages/18/67/36e9267722cc04a6b9f15c7f3441c2363321a3ea07da7ae0c0707beb2a9c/typing_extensions-4.15.0-py3-none-any.whl.metadata
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
   ---------------------------------------- 0


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Loading Blinkit - blinkit_orders.csv into orders table...
✔ Successfully loaded 5000 rows into 'orders'
Loading Blinkit - blinkit_order_items.csv into order_items table...
✔ Successfully loaded 5000 rows into 'order_items'
Loading Blinkit - blinkit_products.csv into products table...
✔ Successfully loaded 268 rows into 'products'
Loading Blinkit - blinkit_customers.csv into customers table...
✔ Successfully loaded 2500 rows into 'customers'
Loading Blinkit - blinkit_marketing_performance.csv into marketing_performance table...
✔ Successfully loaded 5400 rows into 'marketing_performance'
Loading Blinkit - blinkit_customer_feedback.csv into feedback table...
✔ Successfully loaded 5000 rows into 'feedback'
🎯 All tables imported successfully!
